# Model Testing

* Decision Trees
* Random Forest
* SVM
* Ensemble


(*Remember in the presentation to explain what scores that aren't accuracy mean. Show Helen your slides on Monday*)

In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

pd.set_option('display.max_columns', None)
plt.rcParams['figure.facecolor']='w'

import warnings
warnings.filterwarnings('ignore')

In [4]:
#set up train and test

test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_test = test.drop(columns = 'loan_status')
y_test = test.loan_status

X_train = train.drop(columns = 'loan_status')
y_train = train.loan_status

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

## Decision Trees

In [7]:
from sklearn.tree import DecisionTreeClassifier